In [1]:
# Loading Required Packages 
import cobra
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba
import pandas as pd
import numpy as np
import itertools

In [2]:
# Loading the protein data 
df = pd.read_excel(io="ProteinConc.xlsx")

In [3]:
# Loading the model 
%run ModelSetUp.ipynb

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7fa538511950>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7fa537f55e10>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7fa5385119d0>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

Mal_c --> Fum_c + H2O_c
(0.0, 1000.0)
Directionality changed. Dyson et al. 2016 

Mal_c --> 
Fum_c --> 
starch1_h --> 
G6P_c + Pi_h --> G6P_h + Pi_c
G6P_c + Pi_h <=> G6P_h + Pi_c
(-1000.0, 1000.0)
Set to be bidirectional. Dyson et al. 2015 

G6P_c + Pi_h <=> G6P_h + Pi_c
(0.0, 1000.0)
Set as one directional to avoid futile cycles
G6P_c + Pi_h --> G6P_h + Pi_c
(0.0, 1000.0)
Set as one directional to avoid futile cycles
G6P_c + Pi_h --> G6P_h + Pi_c
(0.0, 1000.0)
Set as one directional to avoid futile cycles


In [4]:
# Functions for running the models 

def ATPNADPH(atp,nadph):
    '''Setting the ATP and NADPH flux. Lower bound plus one'''
    NADPH = model.reactions.get_by_id('Fd_DASH_NADPR_h')
    ATPase = model.reactions.get_by_id('ATPase_h')
    NADPH.lower_bound = nadph
    NADPH.upper_bound = nadph + 1.0
    ATPase.lower_bound = atp
    ATPase.upper_bound = atp + 1.0
    print(NADPH)
    print(NADPH.bounds)
    print(ATPase)
    print(ATPase.bounds)
    print(" ")
    
def setobjective(FunName,obj="Maximum"):
    '''Setting the objective function of the model to FunName'''
    model.objective = model.reactions.get_by_id(FunName)
    model.objective_direction = obj
    print(model.objective)
    print("Set as objective function. \n")
    
def constr(reacs,Dat1,Dat2,Dat3,Dat4):
    '''Constraints model reactions for a specific genotype and temperature'''
    for r in reacs:
        upper_vals = []
        lower_vals = []
        for g in r.genes:
            # Get index of protein in data file 
            ind = all_atg.index(g.id)
            # Scaling the values for setting fluxes 
            vals = [i/4000. for i in Dat1[ind],Dat2[ind],Dat3[ind],Dat4[ind]]
            val = np.mean(vals)
            sterr = np.std(vals)/np.sqrt(len(vals))
            upper = val + sterr
            lower = val - sterr
            upper_vals.append(upper)
            lower_vals.append(lower)
        # Constraint flux of forward reaction
        r.upper_bound = round(sum(upper_vals),3)
        # Constraint flux of lower reaction 
        if r.lower_bound < 0.0:
            r.lower_bound = round(-sum(upper_vals),3)
        # No lower bound b/c don't know localization or enzymes are used to in other reactions 

def TPTratio(TPT1,TPT2):
    '''To set TPT1 to TPT2 flux ratios'''
    r = model.reactions.get_by_id('Tr_TPT2')
    r.lower_bound = 0.0
    r.upper_bound = TPT2
    r = model.reactions.get_by_id('Tr_TPT1')
    r.lower_bound = 0.0
    r.upper_bound = TPT1
    #print(cobra.flux_analysis.flux_variability_analysis(model, [NADPH,ATPase]))
    print(" ")
    #inds = (1,101,202,302,401,501,601,701,801,901)
    
def subsys(subsystem):
    for r in model.reactions:
        if r.subsystem == subsystem:
            print(r.reaction)


In [5]:
# Setting the ATP and NADPH production fluxes 
setobjective("FumHA_c")
ATPNADPH(107.0,160.0)
flux_variability_analysis(model, ["Tr_TPT1","Tr_TPT2"])
#subsys("light reactions")

Maximize
-1.0*FumHA_c_reverse_5dd7d + 1.0*FumHA_c
Set as objective function. 

Fd_DASH_NADPR_h: 2.0 Fdrd_h + H_h + NADP_h --> 2.0 Fdox_h + NADPH_h
(160.0, 161.0)
ATPase_h: 3.0 ADP_h + 14.0 H_l + 3.0 Pi_h --> 3.0 ATP_h + 3.0 H2O_h + 11.0 H_h
(107.0, 108.0)
 


,maximum,minimum
Tr_TPT1,928.571429,-35.714286
Tr_TPT2,1000.000000,-857.142857


In [10]:
# Get list of all possible proteins in the model
gene_ids = []
for g in model.genes:
    gene_ids.append(g.id)
# Get list of all measured proteins 
all_atg = []
x=df["Accession"]
for atg in x.values:
    all_atg.append(atg)
# Comparing the measured protein concentrations to those which are in the model 
model_atg = [] # Total measured proteins in model
model_atg_final = [] # Proteins with direct reaction constraint (i.e. single responsible protein)
model_rxns_atg = [] # Model reactions with direct constraints (i.e. single responsible protein)
model_rxns_atgs = [] # Model reactions with multiple possible constraints 
model_atgs_final = [] # Proteins with multiple possible constraints 

for atg in x.values: # Check if protein is in the model 
    if atg in gene_ids:
        model_atg.append(atg)
        # Only parameterize that reaction if that enzyme is the only one responsible for that reaction
        for r in model.genes.get_by_id(atg).reactions:
            if len(r.genes) == 1:
                model_rxns_atg.append(r)
                model_atg_final.append(atg)
            else:
                prot_names = []
                for g in r.genes:
                    prot_names.append(g.id)
                if set(prot_names).issubset(set(all_atg)):
                    model_rxns_atgs.append(r)
                    model_atgs_final = model_atgs_final + prot_names
# Feasible constraint summary 
print("Total measured proteins: {}".format(len(all_atg)))  
print("Total measured proteins in model: {}".format(len(model_atg))) 
print("Total model reactions: {}".format(len(model.reactions)))
print(" ")
# Reactions with simple constraints 
print("Total model reactions with single constraints: {}".format(len(set(model_rxns_atg))))
print("Model proteins for single constraints: {}".format(len(set(model_atg_final))))
print(" ")
# Reactions with multiple possible constraints 
print("Total model reactions with multiple constraints: {}".format(len(set(model_rxns_atgs))))
print("Total model proteins for possible constraints: {}".format(len(set(model_atgs_final))))
# Total constained reactions
constr_reacs = set(model_rxns_atg + model_rxns_atgs) #these are the reactions constrained in the models 
constr_ids = []
for r in constr_reacs:
    constr_ids.append(r.id)

Total measured proteins: 2427
Total measured proteins in model: 245
Total model reactions: 552
 
Total model reactions with single constraints: 62
Model proteins for single constraints: 49
 
Total model reactions with multiple constraints: 39
Total model proteins for possible constraints: 53


In [15]:
# Setting up the Model Constraints
# for different genotypes and temperatures 

def all_constr(C,M,F,S,Cond):
    # Metabolite Constraints 
    f1=0.99*200
    f2=1.01*200
    rMal = model.reactions.get_by_id('Mal_Store')
    rFum = model.reactions.get_by_id('Fum_Store')
    rStarch = model.reactions.get_by_id('Starch_Store')
    rCO2 = model.reactions.get_by_id('Im_CO2')
    FumHA_c = model.reactions.get_by_id('FumHA_c')
    rCO2.upper_bound = C+1.0
    rCO2.lower_bound = C-1.0
    rResp.lower_bound = rCO2.lower_bound*0.2/3
    rResp.upper_bound = 1000.0
    rMal.upper_bound = (M+0.001)*f2
    rMal.lower_bound = (M-0.001)*f1
    rFum.upper_bound = (F+0.001)*f2
    rFum.lower_bound = (F-0.001)*f1
    rStarch.upper_bound = (S-0.001)*f2
    rStarch.lower_bound = (S-0.001)*f1
    for r in [rCO2,rResp,rMal,rFum,rStarch]:
        print(r)
        print(r.bounds)
        print(" ")
    # Proteomic Constraints 
    name1 = "{} 1".format(Cond)
    name2 = "{} 2".format(Cond)
    name3 = "{} 3".format(Cond)
    name4 = "{} 4".format(Cond)
    constr(constr_reacs,df[name1],df[name2],df[name3],df[name4])
    # Loopless Constraints 
    print("For Loopless...")
    setobjective("Im_CO2",obj="Maximum")
    loopless = loopless_solution(model)
    print(loopless["Tr_TPT1"])
    print(loopless["Tr_TPT2"])
    for r in model.reactions:
        if (loopless[r.id] == 0.0):
            r.lower_bound = 0.0
            r.upper_bound = 0.0
        else:
            pass 
    FVA = flux_variability_analysis(model,["Tr_TPT1","Tr_TPT2"])
    modelname = "Arnold2014_{}_Loopless.xml".foramt(Cond)
    cobra.io.write_sbml_model(model,modelname,use_fbc_package=False)
    print("Model saved. Go do flux sampling in MATLAB.")

In [16]:
# Parameterizing the model for Col-0 Control Conditions
all_constr(100,0.003,0.006,0.028,"WT Cntl")
    

Im_CO2:  --> CO2_c
(99.0, 101.0)
 
PyrK_c: ADP_c + H_c + PEP_c --> ATP_c + Pyr_c
(6.6000000000000005, 1000.0)
 
Mal_Store: Mal_c --> 
(0.396, 0.808)
 
Fum_Store: Fum_c --> 
(0.99, 1.414)
 
Starch_Store: starch1_h --> 
(5.346, 5.454)
 
For Loopless...
Maximize
-1.0*Im_CO2_reverse_a674e + 1.0*Im_CO2
Set as objective function. 

17.64515929203541
2.961513274336143


In [9]:


# Col0 Control
rCO2.upper_bound = 101.0
rCO2.lower_bound = 99.0
rResp.lower_bound = rCO2.lower_bound*0.2/3
rResp.upper_bound = 1000.0
rMal.upper_bound = 0.004*f2
rMal.lower_bound = 0.002*f1
rFum.upper_bound = 0.007*f2
rFum.lower_bound = 0.005*f1
rStarch.upper_bound = 0.029*f2
rStarch.lower_bound = 0.027*f1

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
constr(constr_reacs,df["WT Cntl 1"],df["WT Cntl 2"],df["WT Cntl 3"],df["WT Cntl 4"])

# Loopless FBA
model.objective = model.reactions.get_by_id("FumHA_c")
model.objective_direction = "Maximum"
print(model.objective)

loopless = loopless_solution(model)
print(loopless["Tr_TPT1"])
print(loopless["Tr_TPT2"])

for r in model.reactions:
    if (loopless[r.id] == 0.0):
        r.lower_bound = 0.0
        r.upper_bound = 0.0
    else:
        pass 

FVA = flux_variability_analysis(model, model.reactions)
print(FVA.loc["FumHA_c"])
print(FVA.loc["Tr_TPT1"])
print(FVA.loc["Tr_TPT2"])

# Saving the model
# cobra.io.write_sbml_model(model,"Arnold2014_WT_Cntl_Loopless.xml",use_fbc_package=False)

#TPTratio(1000.0,0.0)
#cobra.io.write_sbml_model(model,"Arnold2014_WT_Cntl_tpt2_LooplessC.xml",use_fbc_package=False)
FVA = flux_variability_analysis(model, model.reactions)
print(FVA.loc["FumHA_c"])
# TPTratio(0.0,1000.0)
# cobra.io.write_sbml_model(model,"Arnold2014_WT_Cntl_tpt1_Loopless.xml",use_fbc_package=False)
# FVA = flux_variability_analysis(model, model.reactions)
# print(FVA.loc["FumHA_c"])


NameError: name 'model_rxns_atg' is not defined

In [ ]:
# Parameterizing the model for Fum2 Control Conditions

# Fum2 Control
rCO2.upper_bound = 101.0
rCO2.lower_bound = 99.0
rResp.lower_bound = rCO2.lower_bound*0.2/3
rResp.upper_bound = 1000.0
rMal.upper_bound = 0.013*f2
rMal.lower_bound = 0.011*f1
rFum.upper_bound = 0.0*f2
rFum.lower_bound = 0.0*f1
rStarch.upper_bound = 0.043*f2
rStarch.lower_bound = 0.041*f1

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
r = model.reactions.get_by_id('FumHA_c')
r.upper_bound = 0.0 # Mutant knockout 
r.lower_bound = 0.0 
constr(constr_reacs,df["fum2 Cntl 1"],df["fum2 Cntl 2"],df["fum2 Cntl 3"],df["fum2 Cntl 4"]) #Proteomics 

# Loopless FBA
model.objective = model.reactions.get_by_id("FumHA_c")
model.objective_direction = "Maximum"
print(model.objective)

loopless = loopless_solution(model)
print(loopless["Tr_TPT1"])
print(loopless["Tr_TPT2"])

for r in model.reactions:
    if (loopless[r.id] == 0.0):
        r.lower_bound = 0.0
        r.upper_bound = 0.0
    else:
        pass 

FVA = flux_variability_analysis(model, model.reactions)
print(FVA.loc["FumHA_c"])
print(FVA.loc["Tr_TPT1"])
print(FVA.loc["Tr_TPT2"])

# # Saving the model
# cobra.io.write_sbml_model(model,"Arnold2014_M_Cntl.xml",use_fbc_package=False)

# TPTratio(1000.0,0.0)
cobra.io.write_sbml_model(model,"Arnold2014_M_Cntl_tpt1_LooplessC.xml",use_fbc_package=False)
# TPTratio(0.0,1000.0)
#cobra.io.write_sbml_model(model,"Arnold2014_M_Cntl_tpt1_Loopless.xml",use_fbc_package=False)

In [ ]:
# Parameterizing the model for Col-0 Cold Conditions

# Col0 Cold
rCO2.upper_bound = 101.0
rCO2.lower_bound = 99.0
rResp.lower_bound = rCO2.lower_bound*0.2/3
rResp.upper_bound = 1000.0
rMal.upper_bound = 0.013*f2
rMal.lower_bound = 0.011*f1
rFum.upper_bound = 0.024*f2
rFum.lower_bound = 0.022*f1
rStarch.upper_bound = 0.052*f2
rStarch.lower_bound = 0.05*f1

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
constr(constr_reacs,df["WT cold 1"],df["WT cold 2"],df["WT cold 3"],df["WT cold 4"])

# Loopless FBA
model.objective = model.reactions.get_by_id("FumHA_c")
model.objective_direction = "Maximum"
print(model.objective)

loopless = loopless_solution(model)
print(loopless["Tr_TPT1"])
print(loopless["Tr_TPT2"])

for r in model.reactions:
    if (loopless[r.id] == 0.0):
        r.lower_bound = 0.0
        r.upper_bound = 0.0
    else:
        pass 

FVA = flux_variability_analysis(model, model.reactions)
print(FVA.loc["FumHA_c"])
print(FVA.loc["Tr_TPT1"])
print(FVA.loc["Tr_TPT2"])

# Saving the model
#cobra.io.write_sbml_model(model,"Arnold2014_WT_Cold.xml",use_fbc_package=False)

#TPTratio(1000.0,0.0)
cobra.io.write_sbml_model(model,"Arnold2014_WT_Cold_tpt2_LooplessC.xml",use_fbc_package=False)
#TPTratio(0.0,1000.0)
#cobra.io.write_sbml_model(model,"Arnold2014_WT_Cold_tpt1.xml",use_fbc_package=False)

# for i in range(len(inds)):
#     TPTratio(inds[i],inds[-(i+1)])


In [ ]:
# Parameterizing the model for Fum2 Cold Conditions

# Fum2 Cold
rCO2.upper_bound = 95.0
rCO2.lower_bound = 93.0
rResp.lower_bound = rCO2.lower_bound*0.2/3
rResp.upper_bound = 1000.0
rMal.upper_bound = 0.026*f2
rMal.lower_bound = 0.024*f1
rFum.upper_bound = 0.0*f2
rFum.lower_bound = 0.0*f1
rStarch.upper_bound = 0.064*f2
rStarch.lower_bound = 0.062*f1

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
r = model.reactions.get_by_id('FumHA_c')
r.upper_bound = 0.0 # Mutant knockout 
r.lower_bound = 0.0 
constr(constr_reacs,df["fum2 Cold 1"],df["fum2 Cold 2"],df["fum2 Cold 3"],df["fum2 Cold 4"]) #Proteomics 

# Loopless FBA
model.objective = model.reactions.get_by_id("FumHA_c")
model.objective_direction = "Maximum"
print(model.objective)

loopless = loopless_solution(model)
print(loopless["Tr_TPT1"])
print(loopless["Tr_TPT2"])

for r in model.reactions:
    if (loopless[r.id] == 0.0):
        r.lower_bound = 0.0
        r.upper_bound = 0.0
    else:
        pass 

FVA = flux_variability_analysis(model, model.reactions)
print(FVA.loc["FumHA_c"])
print(FVA.loc["Tr_TPT1"])
print(FVA.loc["Tr_TPT2"])

# Saving the model
#cobra.io.write_sbml_model(model,"Arnold2014_M_Cold.xml",use_fbc_package=False)

#TPTratio(1000.0,0.0)
cobra.io.write_sbml_model(model,"Arnold2014_M_Cold_tpt2_LooplessC.xml",use_fbc_package=False)
#TPTratio(0.0,1000.0)
#cobra.io.write_sbml_model(model,"Arnold2014_M_Cold_tpt1.xml",use_fbc_package=False)

# for i in range(len(inds)):
#     TPTratio(inds[i],inds[-(i+1)])


In [ ]:
# CHRR flux sampling (MATLAB SCRIPT)
# Plotting the overlapping sampling distribution (NEW PYTHON SCRIPT)

In [ ]:
# Deleting model reactions as sets 
req_reacs = []
Nreq_reacs = []

# Exluding reactions which cannot be deleted in combination with others
# Round 1
exp_reacs.append(model.reactions.get_by_id("Tr_O2h"))
# Round 2 
exp_reacs.append(model.reactions.get_by_id("Tr_CO2h"))
# Round 3
exp_reacs.append(model.reactions.get_by_id("Tr_H2Oh"))
exp_reacs.append(model.reactions.get_by_id("Im_NO3"))
exp_reacs.append(model.reactions.get_by_id("Tr_O2m"))
exp_reacs.append(model.reactions.get_by_id("Tr_H2Oh"))
exp_reacs.append(model.reactions.get_by_id("Tr_O2m"))
exp_reacs.append(model.reactions.get_by_id("Tr_NO2"))
exp_reacs.append(model.reactions.get_by_id("Tr_NTT"))
exp_reacs.append(model.reactions.get_by_id("Tr_AAC"))
exp_reacs.append(model.reactions.get_by_id("Tr_TPT1"))
exp_reacs.append(model.reactions.get_by_id("Tr_TPT2"))
exp_reacs.append(model.reactions.get_by_id("Tr_TPT3"))

noAT = []
for r in model.reactions:
    g = r.genes
    for gen in g:
        if gen.id == '-':
            if r not in exp_reacs:
                noAT.append(r)
                r.upper_bound = 0.0
                r.lower_bound = 0.0
                
print(len(noAT))

print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))

# # Trying to delete as high a set of them as possible 
# for s in set(itertools.combinations(noAT, 4)): #set(itertools.combinations(noAT, 1))
#     print("Check")
#     b = []
#     for i in range(len(s)):
#         b.append(s[i].bounds)
#         s[i].upper_bound = 0.0
#         s[i].lower_bound = 0.0
#     interest_reacs = ["Im_CO2","Mal_Store","Fum_Store","Starch_Store","Ex_Suc"]
#     test = []
#     for r in interest_reacs:
#         model.objective = r
#         try:
#             bounds = flux_variability_analysis(model, r)
#             if bounds['minimum'][0] > 0.0:
#                 test.append(0) # Carbon storage ok
#             else:
#                 test.append(1)
#         except Exception:
#             test.append(1)
#     if sum(test) > 0: # Can't store carbon 
#         req_reacs.append(s)
#     else:
#         Nreq_reacs.append(s)
#     for i in range(len(s)):
#         # Resetting the reactions bounds
#         s[i].upper_bound = b[i][1]
#         s[i].lower_bound = b[i][0]
# print("Finished")

In [ ]:
# Deleting model reactions as sets 
req_reacs = []
Nreq_reacs = []

# Trying to delete as high a set of them as possible 
for s in set(itertools.combinations(model.reactions, 1)): #set(itertools.combinations(noAT, 1))
    b = []
    for i in range(len(s)):
        b.append(s[i].bounds)
        s[i].upper_bound = 0.0
        s[i].lower_bound = 0.0
    interest_reacs = ["Im_CO2","Mal_Store","Starch_Store","Ex_Suc"]
    test = []
    for r in interest_reacs:
        model.objective = r
        try:
            bounds = flux_variability_analysis(model, r)
            if bounds['minimum'][0] > 0.0:
                test.append(0) # Carbon storage ok
            else:
                test.append(1)
        except Exception:
            test.append(1)
    if sum(test) > 0: # Can't store carbon 
        req_reacs.append(s)
    else:
        Nreq_reacs.append(s)
    for i in range(len(s)):
        # Resetting the reactions bounds
        s[i].upper_bound = b[i][1]
        s[i].lower_bound = b[i][0]

In [ ]:
rObj = model.reactions.get_by_id('Im_CO2')
print(rObj.reaction)
model.objective = rObj
model.objective_direction = "Maximum"
print(model.objective)
print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))

rObj = model.reactions.get_by_id('ATPase_h')
print(rObj.reaction)
model.objective = rObj
model.objective_direction = "Minimum"
print(model.objective)
print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))

rObj = model.reactions.get_by_id('Fd_DASH_NADPR_h')
print(rObj.reaction)
model.objective = rObj
model.objective_direction = "Maximum"
print(model.objective)
print(cobra.flux_analysis.flux_variability_analysis(model, all_reac))

In [ ]:
# Appending transport reactions which are not to be deleted
r1 = model.reactions.get_by_id("Tr_TPT1")
r2 = model.reactions.get_by_id("Tr_TPT2")

r = r1
r.upper_bound = 0.0
r.lower_bound = 0.0


# Setting reactions without AT codes to zero
noAT = []
for r in model.reactions:
    g = r.genes
    for gen in g:
        if gen.id == '-':
            if r not in exp_reacs:
#                 if r.id[0] is not "T":
#                     print(r.id)
#                     print(r.reaction)
#                     r.upper_bound = 0.0
#                     r.lower_bound = 0.0
                noAT.append(r)
print(len(noAT))
